## 3. 3D Image Registration with Masks

### Registration

Often, some content in the images may not correspond. For example, there may be background content or noisy areas. A mask defined on the fixed and/or moving image can be used to exclude these regions at a pixel level from the similarity metric computations. This improves the robustness of registration.

For more information, see Section 5.4, "Masks" of the [Elastix Manual](http://elastix.isi.uu.nl/download/elastix-5.0.0-manual.pdf).

In [5]:
# First two imports are currently necessary to run ITKElastix on MacOs
from itk import itkElastixRegistrationMethodPython
from itk import itkTransformixFilterPython
import itk
import numpy as np
# import napari

The function calls in the 3D case to import and register the images is similar to the 2D case. Masks, usually binary images, are import with the itk library similar to the images.  

In [6]:
# Import Images
fixed_image = itk.imread('data/CT_3D_lung_fixed.mha', itk.F)
moving_image = itk.imread('data/CT_3D_lung_moving.mha', itk.F)

# Import Custom Parameter Map
parameter_object = itk.ParameterObject.New()
parameter_object.AddParameterFile('data/parameters.3D.NC.affine.ASGD.001.txt')

# Import Mask Images
fixed_mask = itk.imread('data/CT_3D_lung_fixed_mask.mha', itk.UC)
moving_mask = itk.imread('data/CT_3D_lung_moving_mask.mha', itk.UC)

Registration can either be done in one line with the registration function...

In [7]:
# Call registration function
result_image, result_transform_parameters = itk.elastix_registration_method(
    fixed_image, moving_image,
    parameter_object=parameter_object,
    fixed_mask=fixed_mask, moving_mask=moving_mask,
    log_to_console=False)

.. or by initiating an elastix image filter object.

In [10]:
# Load Elastix Image Filter Object
# For now in the 3D case the object oriented method needs explicit specification 
# of the elastix class that should be used
elastix_object = itk.itkElastixRegistrationMethodPython.itkElastixRegistrationMethodIF3IF3.New()

elastix_object.SetFixedImage(fixed_image)
elastix_object.SetMovingImage(moving_image)
elastix_object.SetFixedMask(fixed_mask)
elastix_object.SetMovingMask(moving_mask)
elastix_object.SetParameterObject(parameter_object)

# Set additional options
elastix_object.SetLogToConsole(False)

# Update filter object (required)
elastix_object.UpdateLargestPossibleRegion()

# Results of Registration
result_image = elastix_object.GetOutput()
result_transform_parameters = elastix_object.GetTransformParameterObject()

### Visualization (not working yet on binder)

The external Napari viewer can also be used to view 3D images. The default settings are not ideal for viewer 3D images, however 3D view can be toggled in the viewer interface.

In [1]:
# Cast images to numpy arrays for Napari Viewer
# result_image_np = np.asarray(result_image).astype(np.float32)
# fixed_image_np = np.asarray(fixed_image).astype(np.float32)
# moving_image_np = np.asarray(moving_image).astype(np.float32)

In [2]:
# View output with Napari Viewer
# with napari.gui_qt():
#     viewer = napari.Viewer()
#     viewer.add_image(fixed_image_np)
#     viewer.add_image(moving_image_np)
#     viewer.add_image(result_image_np)